In [ ]:
# 練習 pandas 的 dataframe 資料調整和篩選方式

# 先載入資料並做成多日 dataframe

import os
import pandas

path = "D:\Python\爬蟲\動態爬蟲 ( Selenium )\三大法人每日買賣超資料抓取\download_20250805"

files = os.listdir(path)

all_data = []

for file in files : 

    with open(os.path.join(path,file),"r") as item : 

        dataframe = pandas.read_csv(item)

        dataframe["日期"] = file.split(".")[0]

        dataframe["日期"] = pandas.to_datetime(dataframe["日期"])

        dataframe["買賣差額"] = pandas.to_numeric(dataframe["買賣差額"].str.replace(",",""))

        dataframe = dataframe[["日期","單位名稱",'買賣差額']]        

        all_data.append(dataframe)

combine = pandas.concat(all_data)

pivot = combine.pivot(index="日期",columns="單位名稱",values="買賣差額")



# dataframe 的基本篩選方式


# 基本觀念 : 

# dataframe["欄位"] 代表該欄位底下的資料列

# dataframe["欄位"] > 0 代表該欄位底下的所有資料去做比較，並產生布林列

# dataframe[dataframe["欄位"] > 0] 表示對 dataframe 用布林列來篩選符合的列


# 1. 欄位 ( columns ) : 單純用 [] 和 欄位名稱 來做篩選即可

# a. 選擇單一欄位

combine_column = combine["單位名稱"]

print(combine_column)

# b. 選擇多個欄位 : 改用 list

combine_columns = combine[["單位名稱","買賣差額"]]

print(combine_columns)

# c. 移除單一欄位 : 用 drop 方法

combine_remove_column = combine.drop(columns="單位名稱")

print(combine_remove_column)

# d. 移除多個欄位 : 用 drop 方法 + list

combine_remove_columns = combine.drop(columns=["日期","單位名稱"])

print(combine_remove_columns)



# 2. 資料列 ( rows ) : 一樣用 [] , 但需要用欄位 + 條件運算子來產生布林列來挑出需要的資料列

# a. 日期挑選 ( dataframe 要先做 to_datetime() , 且注意日期要用字串表示，而不是直接打日期 )

combine_date_row = combine[combine["日期"]>"2025-08-01"]

print(combine_date_row)

# b. 特定群組挑選 ( 等於要用 == , 因為是比較產生布林值列來篩選資料列 )

combine_group_row = combine[combine["單位名稱"]=="合計"]

print(combine_group_row)

# c. 特定值挑選 ( dataframe 要先做 to_numeric() )

combine_value_row = combine[combine["買賣差額"] > 0]

print(combine_value_row)

# d. 合併條件篩選 ( 要用小括號 和 & 運算子 )

combine_multi_row = combine[(combine["日期"]>"2025-08-01")&(combine["單位名稱"]=="合計")]

print(combine_multi_row)

# e. 合併條件篩選 ( 用 isin , 基本上就是指定多個群組 == [ ... , ... ] 的操作 , 只是不能這樣做所以用 isin )

# 條件篩選就是用條件運算子 , 像是 == , > , < , != ( 排除資料可以用 ) 等等 , 

# isin 也是條件運算子 , 可產生布林值 , 只是代替 == [ ... , ... ] 而已

combine_multi_isin_row = combine[(combine["日期"]>"2025-08-01")&(combine["單位名稱"].isin(["投信","合計"]))]

print(combine_multi_isin_row)



# pivot 的篩選方式


# pivot 的觀念 : 

# pivot 和 dataframe 不同 , dataframe 的就是欄位和底下欄位的資料列 , 

# pivot 是 dataframe 的一個欄位當作 x 軸 , 另一個當作 y 軸 , 再一個為數值表，所以操作會不太一樣 , 

# pivot 是由 index , columns 和 values 所組成 , 都以這三者作為篩選條件的操作變數為主


# 1. 欄位的篩選方式

# 和 dataframe 一樣 , 都是用 [] , 因為 pivot 也是由 dataframe 構成 , 但要注意新索引 , 新欄位是由舊的組成 , 和原 dataframe 不同

# a. 選擇單一欄位 : 

pivot_column = pivot["投信"]

print(pivot_column)

# b. 選擇多個欄位 : 

pivot_columns = pivot[["投信","合計"]]

print(pivot_columns)

# c. 移除單一欄位 : 

pivot_remove_column = pivot.drop(columns="合計")

print(pivot_remove_column)

# d. 移除多個欄位 : 

pivot_remove_columns = pivot.drop(columns=["投信","合計"])

print(pivot_remove_columns)


# 2. 資料列的篩選方式

# 和 dataframe 一樣，用 [] 放 " 篩選條件產生的布林列 " 來挑選符合條件的資料列


# a. 用 index 來篩選資料列 ( 主要 )

pivot_index_row = pivot["2025-08-01":"2025-08-05"]

pivot_index_row_2 = pivot[pivot.index > "2025-08-01"]

print(pivot_index_row)

print(pivot_index_row_2)

# b. 沒有用 columns 來篩選資料列，但可以用 ["欄位"] > 0 這樣篩選某欄位下面資料列符合條件的資料

# ( 其他欄位還是會顯示，只有不符合資料列會消失，基本上還是資料列的篩選操作 )

pivot_column_row = pivot[pivot["合計"] > 0]

print(pivot_column_row)

# c. 沒有用 values 來篩選資料，因為 values 是二維數值矩陣，產生的布林值也是矩陣，沒辦法給 [] 只有一列資料列的陣列使用


# 3. 用 loc[] 函數來同時篩選欄位和資料列

# 兩個參數，第一個是列的布林陣列，放資料列的篩選條件，第二個是欄位的布林陣列，放欄位篩選條件，資料列和欄位的布林陣列都有

# ( 如果只做資料列可以只放一個參數 [] ，只做欄位就要用切片 [:, ] )

# 參數可以是單一標籤，也可以是多個標籤，切片，篩選條件

# 適用 dataframe 也適用 pivot , 但是兩者欄位和資料列結構不同 , 所以要清楚自己用甚麼在做甚麼篩選，不要把欄位篩選產生的布林陣列放到資料列的


# a. dataframe 針對第一個參數的挑選 ( 只能用條件篩選 , 因為第一個參數挑選的是列 , 而 dataframe 挑列只能靠欄位做條件 , 且沒有日期當索引 )

dataframe_loc_row = combine.loc[combine["日期"] > "2025-08-01"]

print(dataframe_loc_row)

# b. dataframe 針對第二個參數的挑選 ( 基本上和原來一樣 , 就是 標籤 或 list )

dataframe_loc_column = combine.loc[:,"日期"]

dataframe_loc_column_2 = combine.loc[:,["單位名稱","買賣差額"]]

print(dataframe_loc_column)

print(dataframe_loc_column_2)

# c. pivot 針對第一個參數的挑選 ( 因為是列，所以就是用 index 的資料來操作，一樣有標籤或 list )

pivot_loc_row = pivot.loc["2025-08-01"]

pivot_loc_row_2 = pivot.loc[["2025-08-01","2025-08-05"]]

pivot_loc_row_3 = pivot.loc["2025-08-01":"2025-08-05"]

pivot_loc_row_4 = pivot.loc[pivot.index > "2025-08-01"]

print(pivot_loc_row)

print(pivot_loc_row_2)

print(pivot_loc_row_3)

print(pivot_loc_row_4)

# d. pivot 針對第二個參數的挑選 ( 因為是欄，所以就是用 columns 的資料來操作 )

pivot_loc_column = pivot.loc[:,"合計"]

pivot_loc_column_2 = pivot.loc[:,["投信","合計"]]

pivot_loc_column_3 = pivot.loc[:,pivot.columns.isin(["投信","合計"])]

print(pivot_loc_column)

print(pivot_loc_column_2)

print(pivot_loc_column_3)

# e. dataframe 和 pivot 的多條件篩選 

# ( 如果要做很多條件篩選就用 dataframe 多做幾次 loc 即可，另外也比較推薦先在 dataframe 做條件篩選再做 pivot，因為像 values 在 pivot 就沒辦法篩選，但在 dataframe 就比較好篩選 ) 

dataframe_loc_multi = combine.loc[(combine["日期"]>"2025-08-01")&(combine["買賣差額"]>0),["日期","單位名稱"]]

print(dataframe_loc_multi)

pivot_loc_multi = pivot.loc[pivot.index>"2025-08-01",pivot.columns.isin(["投信","合計"])]

print(pivot_loc_multi)



0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
0         自營商(自行買賣)
1           自營商(避險)
2                投信
3    外資及陸資(不含外資自營商)
4             外資自營商
5                合計
Name: 單位名稱, dtype: object
             單位名稱         買賣差額
0       自營商(自行買賣)   1316827555
1         自營商(避險)   4056939563
2              投信   1437205201
3  外資及陸資(不含外資自營商)  14112204090
4           外資自營商            0
5              合計  20923176409
0       自營商(自行買賣)   2796495944
1         自營商(避險)   5602641455
2              投信     17825528
3  外資及陸資(不含外資自營商)   9388571808
4           外資自營商            0
5 

In [ ]:
# pandas 其他技巧


# 1. dataframe 和 pivot 一樣都是 dataframe , 只是行和列定義不太一樣，但都有 dataframe.index , dataframe.columns , dataframe.values 

# 可以對 dataframe.columns 和 dataframe.values 做 list , 就能得到欄位名稱和資料的 list 

# ( dataframe 的 dataframe.index 就只是索引，沒甚麼用，pivot 是另一個欄位來當索引，所以 pivot 用 pivot.index 比較有用 )

import pymysql
import pandas

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    database="example",
    password="12345",
    charset="utf8"

)

sql = "SELECT * FROM new_table_2"

result = pandas.read_sql(sql,connection)

print(list(result.columns))

print(list(result.values))


# 2. dataframe 可以用 dataframe.to_numpy 轉成 numpy，有些使用場合會比較方便

# result 原本是 dataframe

print(result)

# 用 dataframe.to_numpy 轉成 numpy，但是 numpy array 沒有 column 和 row label，所以欄位名稱和列索引會不見

result_2 = result.to_numpy()

print(result_2)


['Seq', '年度', '國小補校校數', '國中補校校數', '高級中等學校進修部校數', '國小補校班級數', '國中補校班級數', '高級中等學校進修部班級數']
[array(['1', '2017', '15', '24', '17', '23', '44', '172'], dtype=object), array(['2', '2018', '15', '23', '17', '22', '43', '165'], dtype=object), array(['3', '2019', '16', '22', '17', '24', '36', '155'], dtype=object), array(['4', '2020', '15', '20', '15', '23', '35', '133'], dtype=object), array(['5', '2021', '15', '17', '15', '23', '32', '128'], dtype=object), array(['6', '2022', '14', '16', '14', '20', '29', '120'], dtype=object), array(['7', '2023', '13', '16', '14', '20', '29', '114'], dtype=object), array(['8', '2024', '13', '16', '14', '18', '28', '105'], dtype=object)]
['1', '2017', '15', '24', '17', '23', '44', '172']
['2', '2018', '15', '23', '17', '22', '43', '165']
['3', '2019', '16', '22', '17', '24', '36', '155']
['4', '2020', '15', '20', '15', '23', '35', '133']
['5', '2021', '15', '17', '15', '23', '32', '128']
['6', '2022', '14', '16', '14', '20', '29', '120']
['7', '2023', '13', '1

C:\Users\zxzxa\AppData\Local\Temp\ipykernel_13736\3199896981.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pandas.read_sql(sql,connection)
